# Tutorial Torch

[ライトニングpytorch入門](https://qiita.com/sh-tatsuno/items/42fccff90c98103dffc9)

[PyTorch入門 [公式Tutorial:DEEP LEARNING WITH PYTORCH: A 60 MINUTE BLITZを読む]](https://qiita.com/north_redwing/items/30f9619f0ee727875250) -> 基本こっち主軸

In [1]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々

### Tensor
- pytorchはTensorという型で演算を行う

In [2]:
x=torch.Tensor(5,3)# Construct a tensor directly from data:
print(x)

y=torch.rand(5,3)
print(y)

z = torch.zeros(5,3, dtype=torch.long)
print(z)

tensor([[-2.7417e+13,  3.0691e-41, -2.8707e+12],
        [ 3.0691e-41, -3.4055e+12,  3.0691e-41],
        [-2.4500e+13,  3.0691e-41, -9.7131e-21],
        [ 0.0000e+00,  1.5835e-43,  0.0000e+00],
        [-2.7325e+13,  3.0691e-41, -2.4585e+13]])
tensor([[0.2979, 0.1188, 0.6558],
        [0.9975, 0.4709, 0.4775],
        [0.1576, 0.4557, 0.8044],
        [0.0100, 0.1896, 0.5190],
        [0.5217, 0.7507, 0.9679]])
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


### numpy
- pytorchを利用するには全ての変数はTensorに変換する必要がある

 - Tensor -> numpy ：(Tensorの変数).numpy()

 - numpy -> Tensor :torch.from_numpy(Numpyの変数)

In [3]:
x = np.random.rand(5, 3)
y = torch.from_numpy(x)
z = y.numpy()
print(x)
print(y)
print(z)

[[0.24647075 0.12197757 0.9411779 ]
 [0.3448102  0.51629155 0.97023378]
 [0.76787071 0.04857577 0.31369453]
 [0.45920206 0.9655576  0.05411122]
 [0.89728857 0.05418069 0.625699  ]]
tensor([[0.2465, 0.1220, 0.9412],
        [0.3448, 0.5163, 0.9702],
        [0.7679, 0.0486, 0.3137],
        [0.4592, 0.9656, 0.0541],
        [0.8973, 0.0542, 0.6257]], dtype=torch.float64)
[[0.24647075 0.12197757 0.9411779 ]
 [0.3448102  0.51629155 0.97023378]
 [0.76787071 0.04857577 0.31369453]
 [0.45920206 0.9655576  0.05411122]
 [0.89728857 0.05418069 0.625699  ]]


- 参照渡し注意
- ここで後半に注目しましょう. aから作成したbは, aを変更すると一緒に変更されいます. 参照渡しということです.
- これ結構大事なことです.

In [4]:
a= torch.ones(5)
print(a)
b = a.numpy()
print(b)
a.add_(1)
print(a)
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


### Variable
- Tensor型にしたものを微分対応させるためにはさらにVariable型にする必要がある
 - Variable使用時に自動微分を対応させるには独自関数を使用しないといけない

In [5]:
x = torch.rand(5, 3)
y = Variable(x)
z = torch.pow(y,2) + 2 #y_i**2 + 2
print(x)
print(y)
print(z)

tensor([[0.6011, 0.1274, 0.2992],
        [0.5663, 0.7715, 0.6420],
        [0.4517, 0.9517, 0.3666],
        [0.3528, 0.7068, 0.5273],
        [0.1872, 0.3867, 0.9556]])
tensor([[0.6011, 0.1274, 0.2992],
        [0.5663, 0.7715, 0.6420],
        [0.4517, 0.9517, 0.3666],
        [0.3528, 0.7068, 0.5273],
        [0.1872, 0.3867, 0.9556]])
tensor([[2.3614, 2.0162, 2.0895],
        [2.3207, 2.5952, 2.4121],
        [2.2040, 2.9056, 2.1344],
        [2.1245, 2.4996, 2.2780],
        [2.0350, 2.1495, 2.9132]])


### shape
- numpyでも形状は非常に大事でした. それはtensorになっても変わりません.
- pytorchではshapeでもsizeでも同じ結果になります.

In [6]:
x = torch.Tensor([[ 0.1804,  0.5438,  0.2560],
        [-0.9047, -0.3714, -0.7198],
        [-1.8642, -0.3904,  1.0680],
        [ 0.4988,  0.7488, -1.1682],
        [ 0.3947,  0.5058,  1.7056]])
print(x.shape)
print(x.size())

torch.Size([5, 3])
torch.Size([5, 3])


- 基本はnumpy likeなのでスライシングも同じようにできます.

In [7]:
x = torch.from_numpy(np.arange(15).reshape(5,3))
print(x[:,0])

tensor([ 0,  3,  6,  9, 12])


- one-hotベクトルにしたり, ベクトルを1行の行列にしたり, という作業はよくやりますね. このあたりに関しては, .view(,) が使えます.

参照：[【PyTorch】Tensorを操作する関数（transpose、view、reshape）](https://qiita.com/kenta1984/items/d68b72214ce92beebbe2)

In [8]:
xview = x.view(-1,15)
print(xview)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]])


### CUDA Tensors
- cudaを使って計算する場合にやることが簡単です.
- tensorを作成するときに, device=で指定するか, .to(device)にするだけです.

- .to()するときは、 x = x.to() しないと反映されないことに注意！！！

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print(device, type(device))
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")
    z = x + y
    print(z)
    print(z.is_cuda)
    print(z.to("cpu", torch.double).is_cuda)       # ``.to`` can also change dtype together!
    print(z.is_cuda)

cuda <class 'torch.device'>
tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15]], device='cuda:0')
True
False
True


### データの取得
- pytorchで与えるデータはTensor(train), Tensor(target)のようにする必要がある。
- データラベルを同時に変換させる関数としてTensorDatasetがある。
- pytorchのDataLoaderはバッチ処理のみ対応している。

In [10]:
#X_train = x
#X_test = x
#train = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
#train_loader = torch.utils.data.DataLoader(train, batch_size=100, shuffle=True)
#test = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
#test_loader = torch.utils.data.DataLoader(test, batch_size=100, shuffle=True)

### AUTOGRAD: AUTOMATIC DIFFERENTIATION
ニューラルネットワークの基本はBP(逆誤差伝播)です.
すなわち, 微分操作が必要不可欠です.
この章ではその微分操作を自動に行ってくれるAUTOGRADについて見ていきましょう.

---
- torch.Tensorには, .require_grad, .gradというattributeが, .backward()というメソッドを持っています.
- この.require_gradをTrueに設定すれば, このtensorに与えられる演算を全て記憶し, .backward()により, 勾配を計算してくれます. その勾配の計算結果は, .gradが保持しています.
- 計算グラフを構築するということでしょうか.
- モデルのテスト時などの勾配計算が必要ないときは, .detach()メソッドにより解除できます.
- これは, with torch.no_grad()のブロック文のなかで記述しても可能です. メモリの節約にもなります.
- .grad_fnは, 勾配計算のために, 演算に用いた関数を示してくれます.

と書いてみましたが, やってみないとよくわかりませんね.
やってみましょう.

In [11]:
x = torch.ones(2,2, requires_grad=True)
print(x)
y = x + 2
print(y)
print(y.requires_grad)# 入力xのrequires_gradをTrueにしたので, yもTrueになっている
print(y.grad_fn)

z = y*y*3
print(z)
out = z.mean()
print(out)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
True
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


Tutorialの例は少し煩雑なので, 簡単な例で見てみましょうか.

y = e<sup>x</sup>

dy/dx =  e<sup>x</sup>

In [12]:
x = torch.tensor(2.0, requires_grad=True)
print(x)
y = torch.exp(x)
print(y)
y.backward()
print(x.grad)

tensor(2., requires_grad=True)
tensor(7.3891, grad_fn=<ExpBackward>)
tensor(7.3891)


微分操作をしない場合は, with torch.no_grad()を使いましょう.
- 推論時には不要でメモリの無駄遣いになります.

In [13]:
print(x.requires_grad)
print((x**2).requires_grad)
with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


### Neural Network
ニューラルネットワークの一般的なトレーニング手順は次のとおりです。

- いくつかの学習可能なパラメーター（または重み）を持つニューラルネットワークを定義する
- 入力のデータセットを反復処理します
- ネットワークを介して入力を処理する
- 損失を計算します（出力がどれほど正しいか）
- 勾配をネットワークのパラメーターに反映
- ネットワークの重みを更新します。通常は、単純な更新ルールを使用します。

簡潔ですね. これ通り順に作って見ましょう.
なお, この方法で作成したobjectは特別なことをしなくても, autogradに依存した形で記述されいるため,先のような手続きを踏むことなく自動微分が出来ます.

#### Define the network

基本的に, nn.Moduleというクラスを継承する形で作成していきます.  
まずは, __init__()でLayerをメンバ変数に格納していきます.  
よく使うLayerとしては, 
- Affine変換の nn.Linear(in_features, out_features, bias=True), 
- 畳み込みのConv1d(in_channels, out_channels, kernel_size), Conv2d(in_channels, out_channels, kernel_size), 
- PooingのMaxPool1d(kernel_size), MaxPool2d(kernel_size), 
- 活性化関数のReLU(), 
- DropoutのDropout(), Dropout2d などでしょうか.

詳しくは, torch.nnの公式documentを参考にLayerを構成していきましょう.

その後は, forward()で順伝播の計算グラフを作成します. この辺りも見たまんまの感じです.

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
net.to("cuda:0")
summary(net, input_size=(1, 32, 32))

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 30, 30]              60
            Conv2d-2           [-1, 16, 13, 13]             880
            Linear-3                  [-1, 120]          69,240
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 81,194
Trainable params: 81,194
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.31
Estimated Total Size (MB

パラメータに関しては, .parameters()で取り出すことができます.

In [15]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


---

それではデータを入力してみましょう.

なぜここで4次元のtensorを入力したのでしょうか?  
それはこの文面を見ればわかります.

torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.
For example, nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width.
If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.

つまり, torch.nnはbatch処理しか受け付けないということです.  
なので, fakeとして1次元でいいのでbatchの形にしてね, と書いてあります

In [16]:
input = torch.randn(1, 1, 32, 32)
input = input.to("cuda")
out = net(input)
print(out)

tensor([[ 0.1293, -0.0203,  0.0446,  0.0295, -0.1021,  0.0275,  0.0100,  0.0721,
         -0.0710,  0.0974]], device='cuda:0', grad_fn=<AddmmBackward>)


---

その後は, 勾配を初期化し, 逆誤差伝播します.  
.backward()メソッドを呼び出すことで, その変数のgradに勾配が格納されます.

まだ損失関数をこの段階では定義していないので, これを計算しても(Errorは出ませんが), 返り値はNoneになります.

In [17]:
net.zero_grad()
#out.backward(torch.randn(1, 10))

#### Loss Function

それでは勾配を計算できるように損失関数を定義してみましょう.  
pytorchでは, torch.nnかtorch.nn.functionalにある損失関数を使うのが一般的です.

In [18]:
output = net(input)
target = torch.randn(10, device="cuda")  # a dummy target, for example
target = target.view(1, -1) #it thesame shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)


tensor(1.2651, device='cuda:0', grad_fn=<MseLossBackward>)


#### Backprop

In [19]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0074,  0.0064,  0.0017,  0.0170,  0.0131,  0.0078], device='cuda:0')


#### Update the weights
勾配を計算しただけで満足してはNeural Networkは学習しません. updateしましょう.

パラメータの更新方法には, SGD, Adam, Momentumなど複数ありますね.  
pytorchでは, torch.optimを使って更新を行うobjectを生成し, optimizer.step()で更新を実行します.

In [21]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update